In [43]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import sklearn.naive_bayes as nb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import TimeDistributed


target_folder = 'swing'
mode = 'X_acc_analysis model'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in tqdm(filename_in_dir):
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
print('Done....')
print(len(arr_label))

  0%|          | 18/12331 [00:00<01:10, 173.45it/s]

this model is  X_acc_analysis model 


load_file_complete
make DataFrame...


100%|██████████| 12331/12331 [01:10<00:00, 175.38it/s]


make DataFrame complete ....
label_encoding processing...
Done....
12331


In [6]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(180, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.2))
    model.add(Conv1D(128,
                     2,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 9s 959us/sample - loss: 0.4991 - acc: 0.8114 - val_loss: 0.2175 - val_acc: 0.9291
Epoch 2/100
9864/9864 [==============================] - 9s 890us/sample - loss: 0.1899 - acc: 0.9387 - val_loss: 0.1434 - val_acc: 0.9570
Epoch 3/100
9864/9864 [==============================] - 9s 885us/sample - loss: 0.1389 - acc: 0.9572 - val_loss: 0.1353 - val_acc: 0.9542
Epoch 4/100
9864/9864 [==============================] - 9s 883us/sample - loss: 0.1038 - acc: 0.9698 - val_loss: 0.1096 - val_acc: 0.9668
Epoch 5/100
9864/9864 [==============================] - 9s 882us/sample - loss: 0.0884 - acc: 0.9724 - val_loss: 0.0768 - val_acc: 0.9777
Epoch 6/100
9864/9864 [==============================] - 9s 881us/sample - loss: 0.0742 - acc: 0.9768 - val_loss: 0.0644 - val_acc: 0.9789
Epoch 7/100
9864/9864 [==============================] - 9s 881us/sample - loss: 0.069

In [10]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(180, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.2))
    model.add(Conv1D(128,
                     2,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(512))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 10s 987us/sample - loss: 0.5140 - acc: 0.8019 - val_loss: 0.4327 - val_acc: 0.8610
Epoch 2/100
9864/9864 [==============================] - 9s 901us/sample - loss: 0.2282 - acc: 0.9249 - val_loss: 0.1547 - val_acc: 0.9566
Epoch 3/100
9864/9864 [==============================] - 9s 903us/sample - loss: 0.1501 - acc: 0.9516 - val_loss: 0.1668 - val_acc: 0.9441
Epoch 4/100
9864/9864 [==============================] - 9s 899us/sample - loss: 0.1202 - acc: 0.9625 - val_loss: 0.1002 - val_acc: 0.9655
Epoch 5/100
9864/9864 [==============================] - 9s 906us/sample - loss: 0.0919 - acc: 0.9708 - val_loss: 0.0987 - val_acc: 0.9680
Epoch 6/100
9864/9864 [==============================] - 9s 907us/sample - loss: 0.0886 - acc: 0.9713 - val_loss: 0.1179 - val_acc: 0.9688
Epoch 7/100
9864/9864 [==============================] - 9s 906us/sample - loss: 0.07

In [13]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(180, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.2))
    model.add(Conv1D(512,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(512))
    model.add(Dropout(0.8))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

s_total_data = []
s_total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    s_total_data.append(tmp_li)
    s_total_label.append(label)
    
s_total_data = np.array(s_total_data)
nsamples, nx, ny = s_total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(s_total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == s_total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(s_total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 10s 1ms/sample - loss: 0.5630 - acc: 0.7920 - val_loss: 0.2542 - val_acc: 0.9177
Epoch 2/100
9864/9864 [==============================] - 9s 940us/sample - loss: 0.2396 - acc: 0.9241 - val_loss: 0.1875 - val_acc: 0.9392
Epoch 3/100
9864/9864 [==============================] - 9s 943us/sample - loss: 0.1685 - acc: 0.9508 - val_loss: 0.1348 - val_acc: 0.9562
Epoch 4/100
9864/9864 [==============================] - 9s 944us/sample - loss: 0.1454 - acc: 0.9558 - val_loss: 0.0943 - val_acc: 0.9716
Epoch 5/100
9864/9864 [==============================] - 9s 945us/sample - loss: 0.1093 - acc: 0.9666 - val_loss: 0.0909 - val_acc: 0.9745
Epoch 6/100
9864/9864 [==============================] - 9s 947us/sample - loss: 0.1019 - acc: 0.9696 - val_loss: 0.1015 - val_acc: 0.9708
Epoch 7/100
9864/9864 [==============================] - 9s 945us/sample - loss: 0.1168

In [20]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(60, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.8))
    model.add(Conv1D(512,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=3))
    model.add(MaxPooling1D(pool_size=3))
    model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
    model.add(Dropout(0.8))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=256,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

s_total_data = []
s_total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    s_total_data.append(tmp_li)
    s_total_label.append(label)
    
s_total_data = np.array(s_total_data)
nsamples, nx, ny = s_total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(s_total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == s_total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(s_total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 4s 375us/sample - loss: 1.3496 - acc: 0.4593 - val_loss: 0.7440 - val_acc: 0.7454
Epoch 2/100
9864/9864 [==============================] - 2s 207us/sample - loss: 0.6741 - acc: 0.7617 - val_loss: 0.3938 - val_acc: 0.8687
Epoch 3/100
9864/9864 [==============================] - 2s 208us/sample - loss: 0.4636 - acc: 0.8473 - val_loss: 0.3052 - val_acc: 0.9051
Epoch 4/100
9864/9864 [==============================] - 2s 208us/sample - loss: 0.3686 - acc: 0.8875 - val_loss: 0.2556 - val_acc: 0.9185
Epoch 5/100
9864/9864 [==============================] - 2s 209us/sample - loss: 0.3181 - acc: 0.9089 - val_loss: 0.2322 - val_acc: 0.9274
Epoch 6/100
9864/9864 [==============================] - 2s 213us/sample - loss: 0.2853 - acc: 0.9203 - val_loss: 0.2320 - val_acc: 0.9287
Epoch 7/100
9864/9864 [==============================] - 2s 209us/sample - loss: 0.265

testing new data result :
[answer]  -->  [predict err]
back_smash  -->  back_cut         err_index number :  73
back_smash  -->  back_cut         err_index number :  74
back_smash  -->  back_cut         err_index number :  75
back_smash  -->  back_cut         err_index number :  77
back_smash  -->  back_cut         err_index number :  84
back_smash  -->  back_cut         err_index number :  85
back_smash  -->  back_cut         err_index number :  86
hit:  80  miss :  7 percent :  91.95402298850574


In [29]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(60, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.3))
    model.add(Conv1D(256,
                     1,
                     padding='valid',
                     activation='relu',
                     strides=3))
    model.add(MaxPooling1D(pool_size=3))
    model.add(LSTM(256,dropout=0.2, recurrent_dropout=0.4))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=256,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

s_total_data = []
s_total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    s_total_data.append(tmp_li)
    s_total_label.append(label)
    
s_total_data = np.array(s_total_data)
nsamples, nx, ny = s_total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(s_total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == s_total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(s_total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 4s 412us/sample - loss: 1.1463 - acc: 0.5444 - val_loss: 0.5276 - val_acc: 0.8358
Epoch 2/100
9864/9864 [==============================] - 2s 211us/sample - loss: 0.4086 - acc: 0.8624 - val_loss: 0.2779 - val_acc: 0.9173
Epoch 3/100
9864/9864 [==============================] - 2s 206us/sample - loss: 0.2781 - acc: 0.9107 - val_loss: 0.2012 - val_acc: 0.9469
Epoch 4/100
9864/9864 [==============================] - 2s 206us/sample - loss: 0.2374 - acc: 0.9263 - val_loss: 0.1870 - val_acc: 0.9408
Epoch 5/100
9864/9864 [==============================] - 2s 207us/sample - loss: 0.1892 - acc: 0.9404 - val_loss: 0.1668 - val_acc: 0.9501
Epoch 6/100
9864/9864 [==============================] - 2s 207us/sample - loss: 0.1753 - acc: 0.9470 - val_loss: 0.1522 - val_acc: 0.9566
Epoch 7/100
9864/9864 [==============================] - 2s 208us/sample - loss: 0.155

In [34]:
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(60, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.3))
    model.add(Conv1D(256,
                     1,
                     padding='valid',
                     activation='relu',
                     strides=3))
    model.add(MaxPooling1D(pool_size=3))
    model.add(LSTM(256,dropout=0.2, recurrent_dropout=0.4))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=256,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

s_total_data = []
s_total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    s_total_data.append(tmp_li)
    s_total_label.append(label)
    
s_total_data = np.array(s_total_data)
nsamples, nx, ny = s_total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(s_total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == s_total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(s_total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 4s 444us/sample - loss: 1.0869 - acc: 0.5747 - val_loss: 0.4904 - val_acc: 0.8439
Epoch 2/100
9864/9864 [==============================] - 2s 210us/sample - loss: 0.4183 - acc: 0.8499 - val_loss: 0.2953 - val_acc: 0.8987
Epoch 3/100
9864/9864 [==============================] - 2s 212us/sample - loss: 0.3022 - acc: 0.9016 - val_loss: 0.2239 - val_acc: 0.9331
Epoch 4/100
9864/9864 [==============================] - 2s 210us/sample - loss: 0.2316 - acc: 0.9272 - val_loss: 0.1991 - val_acc: 0.9380
Epoch 5/100
9864/9864 [==============================] - 2s 211us/sample - loss: 0.1965 - acc: 0.9395 - val_loss: 0.1635 - val_acc: 0.9501
Epoch 6/100
9864/9864 [==============================] - 2s 211us/sample - loss: 0.1758 - acc: 0.9469 - val_loss: 0.1454 - val_acc: 0.9534
Epoch 7/100
9864/9864 [==============================] - 2s 213us/sample - loss: 0.161

In [59]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import sklearn.naive_bayes as nb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import TimeDistributed


target_folder = 'swing'
mode = 'X_acc_analysis model'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in tqdm(filename_in_dir):
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(((df['AX'][i]/1000)**2 + (df['AY'][i]/1000)**2 + (df['AZ'][i]/1000)**2)**0.5 )
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
print('Done....')
print(len(arr_label))

x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))

#####here to change####
######################################################################


rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [10,15,20,30,40,50,100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8,9,10,11,12],
    'criterion' :['gini', 'entropy']
}

mod = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10,n_jobs=-1)

######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(((df['AX'][i]/1000)**2 + (df['AY'][i]/1000)**2 + (df['AZ'][i]/1000)**2)**0.5 )
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

  0%|          | 18/12331 [00:00<01:11, 172.87it/s]

this model is  X_acc_analysis model 


load_file_complete
make DataFrame...


100%|██████████| 12331/12331 [01:11<00:00, 172.99it/s]


make DataFrame complete ....
label_encoding processing...
Done....
12331
fitting  X_acc_analysis model  is complete...
X_acc_analysis model score is : 0.8706931495743818
Validation Result...
hit:  2148  miss :  319


now test new data..
testing new data result :
[answer]  -->  [predict err]
fo_drive  -->  fo_smash err_index :  11
back_short  -->  back_smash err_index :  65
back_smash  -->  fo_cut err_index :  84
hit:  84  miss :  3


In [55]:
df['AX'][1]

1277.0

In [62]:
target_folder = 'swing'
mode = 'X_acc_analysis model'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in tqdm(filename_in_dir):
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(((df['AX'][i]/1000)**2 + (df['AY'][i]/1000)**2 + (df['AZ'][i]/1000)**2)**0.5 )
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
print('Done....')
print(len(arr_label))

x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.9 
session = tf.Session(config=config)

earlystopping = EarlyStopping(monitor='val_loss',patience=10)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(120, input_shape=(60,1),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.3))
    model.add(Conv1D(256,
                     1,
                     padding='valid',
                     activation='relu',
                     strides=3))
    model.add(MaxPooling1D(pool_size=3))
    model.add(LSTM(256,dropout=0.2, recurrent_dropout=0.4))
    model.add(Dropout(0.3))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=256,callbacks=[earlystopping] ,validation_data=(x_test, y_test))
    model.save('model_x.h5')
    model.save_weights('model_x_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

s_total_data = []
s_total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(((df['AX'][i]/1000)**2 + (df['AY'][i]/1000)**2 + (df['AZ'][i]/1000)**2)**0.5 )
        tmp_li.append(tmp)
    
    s_total_data.append(tmp_li)
    s_total_label.append(label)
    
s_total_data = np.array(s_total_data)
nsamples, nx, ny = s_total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(s_total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == s_total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(s_total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))

  0%|          | 18/12331 [00:00<01:09, 176.20it/s]

this model is  X_acc_analysis model 


load_file_complete
make DataFrame...


100%|██████████| 12331/12331 [01:10<00:00, 175.50it/s]


make DataFrame complete ....
label_encoding processing...
Done....
12331
1 Physical GPUs, 1 Logical GPUs
Train on 9864 samples, validate on 2467 samples
Epoch 1/100
9864/9864 [==============================] - 5s 466us/sample - loss: 1.8279 - acc: 0.2233 - val_loss: 1.7050 - val_acc: 0.2769
Epoch 2/100
9864/9864 [==============================] - 2s 204us/sample - loss: 1.5589 - acc: 0.3639 - val_loss: 1.5206 - val_acc: 0.3912
Epoch 3/100
9864/9864 [==============================] - 2s 201us/sample - loss: 1.3671 - acc: 0.4532 - val_loss: 1.1243 - val_acc: 0.5805
Epoch 4/100
9864/9864 [==============================] - 2s 202us/sample - loss: 1.0352 - acc: 0.5975 - val_loss: 0.8792 - val_acc: 0.6737
Epoch 5/100
9864/9864 [==============================] - 2s 203us/sample - loss: 0.8769 - acc: 0.6523 - val_loss: 0.7794 - val_acc: 0.6769
Epoch 6/100
9864/9864 [==============================] - 2s 200us/sample - loss: 0.7749 - acc: 0.6944 - val_loss: 0.7047 - val_acc: 0.7300
Epoch 7/100
9

testing new data result :
[answer]  -->  [predict err]
fo_drive  -->  fo_short         err_index number :  9
fo_drive  -->  fo_smash         err_index number :  11
fo_drive  -->  fo_short         err_index number :  12
back_smash  -->  fo_smash         err_index number :  73
back_smash  -->  back_drive         err_index number :  74
back_smash  -->  back_drive         err_index number :  81
back_smash  -->  fo_short         err_index number :  84
hit:  80  miss :  7 percent :  91.95402298850574
